**1**. (100 points)

In this exercise you will use Spark to build and run a machine learning pipeline to separate 'ham' from 'spam' in SMS text messages. Then you will use the pipeline to classify SMS texts.

- Create a Pandas DataFraem form the data in the file`SMSSpamCollection` where each line is tab separated into (label, text). If you find that the read_xxx function in Pandas does not do the job correctly, read in the file line by line before converting to a DataFrame. Create an index column so that each row has a unique number id.
- Convert to a Spark DataFrame that has two columns (klass, SMS) and split into test and training data sets with proportions 0.8 and 0.2 respectively using a random seed of 123.
- Build a Spark ML pipeline consisting of the following 
    - StringIndexer: To convert `klass` into a numeric `labels` column
    - Tokenizer: To covert `SMS` into a list of tokens
    - StopWordsRemover: To remove "stop words" from the tokens
    - CountVectorizer: To count words (use a vocabular size of 100 and minimum number of occureences of 2)
    - LogisticRegression: Use `maxIter=10`, `regParam=0.001`

- Train the model on the test data.
- Evaluate the precision, recall and accuracy of this model on the test data.

In [1]:
%%spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1692,application_1572292571909_0173,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.clustering import GaussianMixture

from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception AttributeError: "'BinaryClassificationMetrics' object has no attribute '_sc'" in <bound method BinaryClassificationMetrics.__del__ of <pyspark.mllib.evaluation.BinaryClassificationMetrics object at 0x7fdbba76d0d0>> ignored

Load the dataset and create a spark dataframe

In [39]:
url="https://raw.githubusercontent.com/cliburn/bios-823-2019/master/homework/SMSSpamCollection"
data = pd.read_csv(url,"\t", header=None, names=["label","text"])
data.reset_index(inplace=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
cols = ['klass', 'SMS']
df = spark.createDataFrame(data[['label', 'text']], cols)
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[klass: string, SMS: string]

Split the code in train and test:

In [153]:
train, test = df.randomSplit([0.8, 0.2], seed=123)
train.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[klass: string, SMS: string]

In [154]:
type(test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [155]:
train.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|klass|                 SMS|
+-----+--------------------+
|  ham| said kiss, kiss,...|
|  ham|   10 min later k...|
|  ham|        26th OF JULY|
|  ham|4 oclock at mine....|
|  ham|7 at esplanade.. ...|
+-----+--------------------+
only showing top 5 rows

Build a ML pipeline

In [156]:
indexer = StringIndexer(
    inputCol="klass", 
    outputCol="labels"
)

tokenizer = Tokenizer(
    inputCol="SMS",
    outputCol="tokens"
)

remover = StopWordsRemover(
    inputCol="tokens",
    outputCol="filtered"
)

CountVectorizerModel = CountVectorizer(
    inputCol="filtered",
    outputCol="features",
    vocabSize=100,
    minDF=2
)

lr = LogisticRegression(
    featuresCol="features", 
    labelCol="labels",
    maxIter=10,
    regParam=0.001
)

pipeline = Pipeline(stages=[indexer, tokenizer, remover, CountVectorizerModel,lr])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Let's train and evaluate the model

In [157]:
model = pipeline.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    prediction = model.transform(test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
score = prediction.select(['labels', 'prediction'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
score.show(n=score.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+
|labels|prediction|
+------+----------+
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       1.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|


In [177]:
tp = float(score.rdd.map(lambda x: x[0]==1 and x[1]==1).sum())
tn = float(score.rdd.map(lambda x: x[0]==0 and x[1]==0).sum())
fp = float(score.rdd.map(lambda x: x[0]==0 and x[1]==1).sum())
fn = float(score.rdd.map(lambda x: x[0]==1 and x[1]==0).sum())
p = float(score.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [183]:
print('Accuracy = %s' % ((tp+tn)/p))
print('Recall = %s' % (tp/(tp+fn)))
print('Precision = %s' % (tp/(tp+fp)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.953612845674
Recall = 0.751724137931
Precision = 0.872

**2** (100 points)

In this exercise, you will simulate running a machine learning pipeline to classify steaming data.

- Convert the test DataFrame into a Pandas DataFrame
- Write each row of the DataFrame to a separate tab-delimited file in a folder called "incoming_sms"
- Create a Structured Streaming DataFrame using `readStream` with `option("maxFilesPerTrigger", 1)` to simulate streaming data
- Use the fitted pipeline created in Ex. 1 to transform the input stream
- Write the transformed stream to memory with name `sms_pred
- Sleep 30 seconds
- Use an SQL query to show the `index`, `label` and `prediction` columns
- Sleep 30 more seconds
- Use an SQL query to show the `index`, `label` and `prediction` columns